LangGraph 流式输出

model参数值（传输方式） 
+ values 在图表的每个步骤之后传输状态的完整值
+ updates 将图表每一步之后的更新流式传输到状态
+ custom 从图形节点内部传输自定义数据
+ messages 为调用LLM的图形节点传输LLM令牌和元数据
+ debug 在图表的整个执行过程中传输尽可能多的信息

In [7]:
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict
from langgraph.graph import StateGraph, START
from dotenv import load_dotenv

load_dotenv(".env", override=True)

model = ChatOpenAI(
    model=os.environ.get("DEEPSEEK_MODEL"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    temperature=0.0,
)


class State(TypedDict):
    topic: str
    joke: str


def refine_topic(state: State):
    return {"topic": state["topic"] + " 和小狗"}


def genrate_joke(state: State):
    response = model.invoke(
        [{"role": "user", "content": f"生成一个关于{state['topic']}的笑话"}]
    )
    return {"joke": response.content}


graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(genrate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "genrate_joke")
    .compile()
)

### stream_mode="values"

In [4]:
for chunk in graph.stream({"topic": "冰淇淋"}, stream_mode="values"):
    print(chunk)

{'topic': '冰淇淋'}
{'topic': '冰淇淋 和小狗'}
{'topic': '冰淇淋 和小狗', 'joke': '这是一个关于冰淇淋 和小狗的笑话'}


### stream_mode="updates"

In [5]:
for chunk in graph.stream({"topic": "冰淇淋"}, stream_mode="updates"):
    print(chunk)

{'refine_topic': {'topic': '冰淇淋 和小狗'}}
{'genrate_joke': {'joke': '这是一个关于冰淇淋 和小狗的笑话'}}


### stream_mode="debug"

In [6]:
for chunk in graph.stream({"topic": "冰淇淋"}, stream_mode="debug"):
    print(chunk)

{'step': 1, 'timestamp': '2026-01-04T08:55:58.645043+00:00', 'type': 'task', 'payload': {'id': 'd569abe5-71b1-3760-5a89-face404a258f', 'name': 'refine_topic', 'input': {'topic': '冰淇淋'}, 'triggers': ('branch:to:refine_topic',)}}
{'step': 1, 'timestamp': '2026-01-04T08:55:58.645306+00:00', 'type': 'task_result', 'payload': {'id': 'd569abe5-71b1-3760-5a89-face404a258f', 'name': 'refine_topic', 'error': None, 'result': {'topic': '冰淇淋 和小狗'}, 'interrupts': []}}
{'step': 2, 'timestamp': '2026-01-04T08:55:58.645447+00:00', 'type': 'task', 'payload': {'id': 'cc4d447c-6def-d834-7c80-aee98a82d5ef', 'name': 'genrate_joke', 'input': {'topic': '冰淇淋 和小狗'}, 'triggers': ('branch:to:genrate_joke',)}}
{'step': 2, 'timestamp': '2026-01-04T08:55:58.645600+00:00', 'type': 'task_result', 'payload': {'id': 'cc4d447c-6def-d834-7c80-aee98a82d5ef', 'name': 'genrate_joke', 'error': None, 'result': {'joke': '这是一个关于冰淇淋 和小狗的笑话'}, 'interrupts': []}}


### stream_mode="messages"

In [8]:
for message_chunk, metadata in graph.stream(
    {"topic": "冰淇淋"}, stream_mode="messages"
):
    print(message_chunk.content, end="|", flush=True)

|冰淇淋|车|刚|停|稳|，|一只|小狗|就|冲|过来|，|尾巴|摇|得像|螺旋|桨|。

|主人|赶紧|拉住|它|：“|不行|！|你|吃了|巧克力|味|冰淇淋|会|生|病的|！”

|小狗|一|屁股|坐下|，|眼|巴巴|盯着|甜|筒|。|卖|冰淇淋|的大|叔|笑了|：“|它|每周|都|来|，|其实|只|想要|这个|——”

|说着|递|过来|一个|**|空|蛋|筒|**|。

|小狗|立刻|叼|住|，|蹦|蹦|跳|跳|走了|，|蛋|筒|套|在|鼻|子上|像|个小|喇叭|。

|主人|恍然大悟|：“|原来|它|要|的不是|冰淇淋|…|是|鼻子|上的|新|玩具|！”

|大叔|眨|眨眼|：“|上周|它|用|这个|装|了我的|午饭|肉|丸|呢|。”|||